## Battle of the Neighborhoods:
## A City Analysis Report: A comparison of Brooklyn and Queens, NY

Dr.Ahmet TARTAR

## Introduction

New York is one of the largest metropolitan cities in the world. New York City (NYC), also known as the City of New York or simply New York (NY), is the most populous city in the United States. With an estimated 2018 population of 8,398,748 distributed over a land area of about 302.6 square miles (784 km2), New York is also the most densely populated major city in the United States [1]. It is one of the cities where people from all over the world want to visit or live.

New York City encompasses five county-level administrative divisions called boroughs: The Bronx, Brooklyn, Manhattan, Queens, and Staten Island. Each borough is coterminous with a respective county of New York State. The boroughs of Queens and the Bronx are concurrent with the counties of the same name, while the boroughs of Manhattan, Brooklyn, and Staten Island correspond to New York, Kings, and Richmond counties [2].

**Brooklyn:** Brooklyn (Kings County), on the western tip of Long Island, is the city's most populous borough. Brooklyn is known for its cultural, social, and ethnic diversity, an independent art scene, distinct neighborhoods, and a distinctive architectural heritage. Downtown Brooklyn is the largest central core neighborhood in the outer boroughs [3]. 

 


**Queens (Queens County)**, on Long Island north and east of Brooklyn, is geographically the largest borough, the most ethnically diverse county in the United States, and the most ethnically diverse urban area in the world [4].

 




## Problem and Purpose

In the light of these information, it is very important to have information about these boroughs for those who want to move from another country of the world, or to relocate to a new city, or to start a new business.
In general, when we think about the best place to live, to relocate to a new business, a lot of parameters should be considered when trying to make a comparison between cities, towns, or neighborhoods. Here is some reasons:

- **Overall Comparison:** This is a comparison of the same factors for each city. Some of the popular factors include population, cost of living, average rent, crime rate, tax rates, and air quality.
- **Crime Rates:** The statistics are two cities, then measures them against national statistics.
- **Comparison of Costs of Living and Salary:** Some factors for this comparison include statistics on food, housing, utilities, transportation and more. This is a useful way to find out if your salary will measure up in the new city.
- **Compare Schools:** This is the best school in the area. It mostly takes into consideration the test scores and the student ratios, including the teacher's experience.
- **Neighborhood Comparison:** This is the best place to live in any given city. These sites allow you to see some interesting facts about the various communities.

In this study, we will prepare a city analysis report on the two major borough, Brooklyn and Queens in NY. So we will learn about the similarities and differences about these cities and the characteristics of the cities themselves.
Of course, for a complete evaluation, many of the above parameters should be evaluated. While all of these analyses are useful for comparing the neighborhoods, there is nothing like visiting the actual city, seeing the neighborhoods, and speaking with residents. If it's possible, an in-person visit is highly recommended before making a big move or relocating decision.

## Data

We will convert addresses into their equivalent latitude and longitude values. Also we will use the Foursquare API to explore neighborhoods, Brooklyn and Queens in New York City. You will use the explore function to get the most common venue categories in each neighborhood, and then use this feature to group the neighborhoods into clusters. You will use the k-means clustering algorithm to complete this task. Finally, you will use the Folium library to visualize the neighborhoods in New York City and their emerging clusters.
In order to segment the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the latitude and longitude coordinates of each neighborhood.
Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

## METHODOLOGY

Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

print('Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.1.1d             |       h516909a_0         2.1 MB  conda-forge
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    ca-certificates-2019.11.28 |       hecc5488_0         145 KB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    certifi-2019.11.28         |           py36_0         149 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0         conda-forge
    geopy:           1.20.0-py_0       conda-forge

The following packages will be UPDATED:

    ca-

Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood. 

Luckily, this dataset exists for free on the web. Feel free to try to find this dataset on your own, but here is the link to the dataset: https://geo.nyu.edu/catalog/nyu_2451_34572

For your convenience, I downloaded the files and placed it on the server, so you can simply run a `wget` command and access the data. So let's go ahead and do that.

In [59]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

Data downloaded!


### Load and Explore the New York Data

In [60]:
with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Notice how all the relevant data is in the features key, which is basically a list of the neighborhoods. So, let's define a new variable that includes this data.

In [61]:
neighborhoods_data = newyork_data['features']

Let's take a look at the first item in this list.

In [62]:
neighborhoods_data[0]

{'type': 'Feature',
 'id': 'nyu_2451_34572.1',
 'geometry': {'type': 'Point',
  'coordinates': [-73.84720052054902, 40.89470517661]},
 'geometry_name': 'geom',
 'properties': {'name': 'Wakefield',
  'stacked': 1,
  'annoline1': 'Wakefield',
  'annoline2': None,
  'annoline3': None,
  'annoangle': 0.0,
  'borough': 'Bronx',
  'bbox': [-73.84720052054902,
   40.89470517661,
   -73.84720052054902,
   40.89470517661]}}

### Tranform the data into a pandas dataframe

The next task is essentially transforming this data of nested Python dictionaries into a pandas dataframe. So let's start by creating an empty dataframe.

In [63]:
# define the dataframe columns
column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)

In [64]:
neighborhoods

,Borough,Neighborhood,Latitude,Longitude


In [65]:
for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [66]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


In [67]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 5 boroughs and 306 neighborhoods.


### Use geopy library to get the latitude and longitude values of New York City.

In order to define an instance of the geocoder, we need to define a user_agent. We will name our agent ny_explorer, as shown below.

In [68]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


### Create a map of New York with neighborhoods superimposed on top.

In [69]:
# create map of New York using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

Folium is a great visualization library. Feel free to zoom into the above map, and click on each circle mark to reveal the name of the neighborhood and its respective borough.

However, for illustration purposes, let's simplify the above map and segment and cluster only the neighborhoods in Manhattan. So let's slice the original dataframe and create a new dataframe of the Manhattan data.

## A. BROOKLYN

In [70]:
brooklyn_data = neighborhoods[neighborhoods['Borough'] == 'Brooklyn'].reset_index(drop=True)
brooklyn_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Brooklyn,Bay Ridge,40.625801,-74.030621
1,Brooklyn,Bensonhurst,40.611009,-73.995180
2,Brooklyn,Sunset Park,40.645103,-74.010316
3,Brooklyn,Greenpoint,40.730201,-73.954241
4,Brooklyn,Gravesend,40.595260,-73.973471


In [71]:
address = 'Brooklyn, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brooklyn are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Brooklyn are 40.6501038, -73.9495823.


As we did with all of New York City, let's visualizat Brooklyn the neighborhoods in it.

In [74]:
# create map of Manhattan using latitude and longitude values
map_brooklyn = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(brooklyn_data['Latitude'], brooklyn_data['Longitude'], brooklyn_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_brooklyn)  
    
map_brooklyn

### A1. The Foursquare API

In [75]:
# Define Foursquare Credentials and Version
CLIENT_ID = 'NNKKGPUSITWUJZNV5GJUHKUSPHL1MCGOTTWBDEHCOP01IRGV' # your Foursquare ID
CLIENT_SECRET = 'QM2K10LNRU40U3CAVO1UM1DDAW1BDK0HDZPKDGL2PL1YE3MQ' # your Foursquare Secret
VERSION = '20200108' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NNKKGPUSITWUJZNV5GJUHKUSPHL1MCGOTTWBDEHCOP01IRGV
CLIENT_SECRET:QM2K10LNRU40U3CAVO1UM1DDAW1BDK0HDZPKDGL2PL1YE3MQ


In [25]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

84 venues were returned by Foursquare.


In [80]:
##
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [81]:
brooklyn_venues = getNearbyVenues(names=brooklyn_data['Neighborhood'],
                                   latitudes=brooklyn_data['Latitude'],
                                   longitudes=brooklyn_data['Longitude']
                                  )

Bay Ridge
Bensonhurst
Sunset Park
Greenpoint
Gravesend
Brighton Beach
Sheepshead Bay
Manhattan Terrace
Flatbush
Crown Heights
East Flatbush
Kensington
Windsor Terrace
Prospect Heights
Brownsville
Williamsburg
Bushwick
Bedford Stuyvesant
Brooklyn Heights
Cobble Hill
Carroll Gardens
Red Hook
Gowanus
Fort Greene
Park Slope
Cypress Hills
East New York
Starrett City
Canarsie
Flatlands
Mill Island
Manhattan Beach
Coney Island
Bath Beach
Borough Park
Dyker Heights
Gerritsen Beach
Marine Park
Clinton Hill
Sea Gate
Downtown
Boerum Hill
Prospect Lefferts Gardens
Ocean Hill
City Line
Bergen Beach
Midwood
Prospect Park South
Georgetown
East Williamsburg
North Side
South Side
Ocean Parkway
Fort Hamilton
Ditmas Park
Wingate
Rugby
Remsen Village
New Lots
Paerdegat Basin
Mill Basin
Fulton Ferry
Vinegar Hill
Weeksville
Broadway Junction
Dumbo
Homecrest
Highland Park
Madison
Erasmus


In [82]:
print(brooklyn_venues.shape)
brooklyn_venues.head()

(2788, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Bay Ridge,40.625801,-74.030621,Pilo Arts Day Spa and Salon,40.624748,-74.030591,Spa
1,Bay Ridge,40.625801,-74.030621,Bagel Boy,40.627896,-74.029335,Bagel Shop
2,Bay Ridge,40.625801,-74.030621,Cocoa Grinder,40.623967,-74.030863,Juice Bar
3,Bay Ridge,40.625801,-74.030621,Pegasus Cafe,40.623168,-74.031186,Breakfast Spot
4,Bay Ridge,40.625801,-74.030621,Ho' Brah Taco Joint,40.622960,-74.031371,Taco Place


In [83]:
brooklyn_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Bath Beach,44,44,44,44,44,44
Bay Ridge,84,84,84,84,84,84
Bedford Stuyvesant,25,25,25,25,25,25
Bensonhurst,32,32,32,32,32,32
Bergen Beach,8,8,8,8,8,8
Boerum Hill,94,94,94,94,94,94
Borough Park,21,21,21,21,21,21
Brighton Beach,45,45,45,45,45,45
Broadway Junction,14,14,14,14,14,14


In [84]:
print('There are {} uniques categories.'.format(len(brooklyn_venues['Venue Category'].unique())))

There are 289 uniques categories.


### A2. Analyze Each Neighborhood

In [85]:
# one hot encoding
brooklyn_onehot = pd.get_dummies(brooklyn_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
brooklyn_onehot['Neighborhood'] = brooklyn_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [brooklyn_onehot.columns[-1]] + list(brooklyn_onehot.columns[:-1])
brooklyn_onehot = brooklyn_onehot[fixed_columns]

brooklyn_onehot.head()

,Yoga Studio,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape St

In [86]:
brooklyn_onehot.shape

(2788, 289)

Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [87]:
brooklyn_grouped = brooklyn_onehot.groupby('Neighborhood').mean().reset_index()
brooklyn_grouped

,Neighborhood,Yoga Studio,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Auto Garage,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Baseball Stadium,Basketball Court,Beach,Beer Bar,Beer Garden,Beer Store,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Boutique,Boxing Gym,Breakfast Spot,Brewery,Bridge,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Caucasian Restaurant,Cha Chaan Teng,Check Cashing Service,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,Concert Hall,Convenience Store,Cosmetics Shop,Coworking Space,Creperie,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distillery,Dive Bar,Doctor's Office,Dog Run,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Electronics Store,Ethiopian Restaurant,Event Service,Event Space,Factory,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Golf Course,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gymnastics Gym,Halal Restaurant,Harbor / Marina,Hardware Store,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Herbs & Spices Store,History Museum,Hockey Field,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotpot Restaurant,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Intersection,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Korean Restaurant,Latin American Restaurant,Laundromat,Laundry Service,Lebanese Restaurant,Library,Lingerie Store,Liquor Store,Lounge,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motorcycle Shop,Movie Theater,Moving Target,Museum,Music Store,Music Venue,Nail Salon,New American Restaurant,Nightclub,Non-Profit,Noodle House,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Other Repair Shop,Outdoors & Recreation,Outlet Store,Pakistani Restaurant,Paper / Office Supplies Store,Park,Performing Arts Venue,Peruvian Restaurant,Pet Store,Pharmacy,Photography Studio,Pie Shop,Piercing Parlor,Pilates Studio,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Print Shop,Pub,Racetrack,Ramen Restaurant,Record Shop,Rental Car Location,Residential Building (Apartment / Condo),Restaurant,Rock Club,Roof Deck,Russian Restaurant,Sake Bar,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skating Rink,Ski Area,Smoke Shop,Snack Place,Soccer Field,South American Restaurant,Southern / Soul Food Restaurant,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Stadium,Stationery Store,Steakhouse,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tattoo Parlor,Tea Room,Tennis Court,Thai Restaurant,Theater,Theme Park Ride / Attraction,Thrift / Vintage Store,Tibetan Restaurant,Tiki Bar,Toy / Game Store,Trail,Turkish Restaurant,Used Bookstore,Vape St

In [88]:
brooklyn_grouped.shape

(70, 289)

Let's print each neighborhood along with the top 5 most common venues

In [89]:
num_top_venues = 5

for hood in brooklyn_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = brooklyn_grouped[brooklyn_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bath Beach----
                venue  freq
0            Pharmacy  0.07
1  Chinese Restaurant  0.07
2         Pizza Place  0.05
3     Bubble Tea Shop  0.05
4    Sushi Restaurant  0.05


----Bay Ridge----
                 venue  freq
0                  Spa  0.07
1   Italian Restaurant  0.07
2          Pizza Place  0.04
3  American Restaurant  0.04
4     Greek Restaurant  0.04


----Bedford Stuyvesant----
                 venue  freq
0          Pizza Place  0.08
1                 Café  0.08
2          Coffee Shop  0.08
3                  Bar  0.08
4  Japanese Restaurant  0.04


----Bensonhurst----
                venue  freq
0  Chinese Restaurant  0.09
1      Ice Cream Shop  0.06
2              Bakery  0.06
3  Italian Restaurant  0.06
4          Donut Shop  0.06


----Bergen Beach----
                venue  freq
0     Harbor / Marina  0.25
1          Donut Shop  0.12
2      Baseball Field  0.12
3   Other Repair Shop  0.12
4  Athletics & Sports  0.12


----Boerum Hill----
             

              venue  freq
0       Pizza Place  0.36
1  Video Game Store  0.09
2            Bakery  0.09
3        Bagel Shop  0.09
4       Candy Store  0.09


----Mill Basin----
                 venue  freq
0          Pizza Place  0.09
1                 Bank  0.06
2       Cosmetics Shop  0.06
3   Chinese Restaurant  0.06
4  Japanese Restaurant  0.06


----Mill Island----
                   venue  freq
0                   Pool   1.0
1            Yoga Studio   0.0
2           Optical Shop   0.0
3           Outlet Store   0.0
4  Outdoors & Recreation   0.0


----New Lots----
                 venue  freq
0        Grocery Store  0.11
1          Pizza Place  0.11
2   Chinese Restaurant  0.11
3          Bus Station  0.05
4  Fried Chicken Joint  0.05


----North Side----
                 venue  freq
0          Coffee Shop  0.07
1          Pizza Place  0.05
2          Yoga Studio  0.03
3             Wine Bar  0.03
4  American Restaurant  0.03


----Ocean Hill----
                             ven

### Let's put that into a pandas dataframe
First, let's write a function to sort the venues in descending order.

In [90]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [91]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = brooklyn_grouped['Neighborhood']

for ind in np.arange(brooklyn_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(brooklyn_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bath Beach,Chinese Restaurant,Pharmacy,Fast Food Restaurant,Donut Shop,Bubble Tea Shop,Italian Restaurant,Pizza Place,Sushi Restaurant,Rental Car Location,Kebab Restaurant
1,Bay Ridge,Italian Restaurant,Spa,Pizza Place,Bar,Chinese Restaurant,Thai Restaurant,American Restaurant,Greek Restaurant,Ice Cream Shop,Bagel Shop
2,Bedford Stuyvesant,Pizza Place,Coffee Shop,Café,Bar,BBQ Joint,Playground,Discount Store,Deli / Bodega,Park,Cocktail Bar
3,Bensonhurst,Chinese Restaurant,Sushi Restaurant,Donut Shop,Italian Restaurant,Ice Cream Shop,Bakery,Factory,Noodle House,Cha Chaan Teng,Spa
4,Bergen Beach,Harbor / Marina,Donut Shop,Hockey Field,Other Repair Shop,Athletics & Sports,Baseball Field,Playground,Flower Shop,Flea Market,Fish Market


### A3. Cluster Neighborhoods

Run k-means to cluster the neighborhood into 5 clusters.

In [93]:
# set number of clusters
kclusters = 5

brooklyn_grouped_clustering = brooklyn_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(brooklyn_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 3, 3, 3, 0, 3, 1, 1, 1, 3], dtype=int32)

Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [94]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

brooklyn_merged = brooklyn_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
brooklyn_merged = brooklyn_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

brooklyn_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Brooklyn,Bay Ridge,40.625801,-74.030621,3,Italian Restaurant,Spa,Pizza Place,Bar,Chinese Restaurant,Thai Restaurant,American Restaurant,Greek Restaurant,Ice Cream Shop,Bagel Shop
1,Brooklyn,Bensonhurst,40.611009,-73.995180,3,Chinese Restaurant,Sushi Restaurant,Donut Shop,Italian Restaurant,Ice Cream Shop,Bakery,Factory,Noodle House,Cha Chaan Teng,Spa
2,Brooklyn,Sunset Park,40.645103,-74.010316,1,Mexican Restaurant,Bakery,Bank,Latin American Restaurant,Pizza Place,Mobile Phone Shop,Fried Chicken Joint,Gym,Grocery Store,Sandwich Place
3,Brooklyn,Greenpoint,40.730201,-73.954241,3,Bar,Coffee Shop,Cocktail Bar,Pizza Place,Café,Sushi Restaurant,Grocery Store,Mexican Restaurant,French Restaurant,Yoga Studio
4,Brooklyn,Gravesend,40.595260,-73.973471,3,Chinese Restaurant,Lounge,Pizza Place,Italian Restaurant,Bakery,Baseball Field,Martial Arts Dojo,Metro Station,Breakfast Spot,Sporting Goods Shop


Finally, let's visualize the resulting clusters

In [95]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(brooklyn_merged['Latitude'], brooklyn_merged['Longitude'], brooklyn_merged['Neighborhood'], brooklyn_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### A4. Examine Clusters

In [96]:
#Cluster 1
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 0, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
45,Bergen Beach,Harbor / Marina,Donut Shop,Hockey Field,Other Repair Shop,Athletics & Sports,Baseball Field,Playground,Flower Shop,Flea Market,Fish Market
59,Paerdegat Basin,Harbor / Marina,Food,Bus Station,Asian Restaurant,Women's Store,Filipino Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field


In [97]:
#Cluster 2
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 1, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
2,Sunset Park,Mexican Restaurant,Bakery,Bank,Latin American Restaurant,Pizza Place,Mobile Phone Shop,Fried Chicken Joint,Gym,Grocery Store,Sandwich Place
5,Brighton Beach,Russian Restaurant,Eastern European Restaurant,Restaurant,Beach,Gourmet Shop,Bank,Fast Food Restaurant,Mobile Phone Shop,Sushi Restaurant,Pharmacy
7,Manhattan Terrace,Pizza Place,Donut Shop,Ice Cream Shop,Convenience Store,Bus Station,Spa,Mobile Phone Shop,Steakhouse,Organic Grocery,Chinese Restaurant
8,Flatbush,Pharmacy,Coffee Shop,Caribbean Restaurant,Mexican Restaurant,Deli / Bodega,Bagel Shop,Diner,Pizza Place,Donut Shop,Sandwich Place
11,Kensington,Thai Restaurant,Grocery Store,Pizza Place,Sandwich Place,Pharmacy,Ice Cream Shop,Restaurant,Spa,Donut Shop,Gas Station
25,Cypress Hills,Pizza Place,Fried Chicken Joint,Ice Cream Shop,Fast Food Restaurant,Donut Shop,Latin American Restaurant,Bakery,Food,South American Restaurant,Spanish Restaurant
27,Starrett City,Moving Target,Supermarket,Shopping Mall,Donut Shop,Convenience Store,Pharmacy,American Restaurant,Pizza Place,Caribbean Restaurant,Flower Shop
33,Bath Beach,Chinese Restaurant,Pharmacy,Fast Food Restaurant,Donut Shop,Bubble Tea Shop,Italian Restaurant,Pizza Place,Sushi Restaurant,Rental Car Location,Kebab Restaurant
34,Borough Park,Bank,Pizza Place,Deli / Bodega,Pharmacy,Coffee Shop,Men's Store,Café,Fast Food Restaurant,Farmers Market,Chinese Restaurant
35,Dyker Heights,Dance Studio,Bagel Shop,Burger Joint,Plaza,Golf Course,Filipino Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field


In [98]:
#Cluster 3
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 2, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
30,Mill Island,Pool,Women's Store,Fish & Chips Shop,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field,Filipino Restaurant,Fish Market


In [99]:
#Cluster 4
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 3, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Bay Ridge,Italian Restaurant,Spa,Pizza Place,Bar,Chinese Restaurant,Thai Restaurant,American Restaurant,Greek Restaurant,Ice Cream Shop,Bagel Shop
1,Bensonhurst,Chinese Restaurant,Sushi Restaurant,Donut Shop,Italian Restaurant,Ice Cream Shop,Bakery,Factory,Noodle House,Cha Chaan Teng,Spa
3,Greenpoint,Bar,Coffee Shop,Cocktail Bar,Pizza Place,Café,Sushi Restaurant,Grocery Store,Mexican Restaurant,French Restaurant,Yoga Studio
4,Gravesend,Chinese Restaurant,Lounge,Pizza Place,Italian Restaurant,Bakery,Baseball Field,Martial Arts Dojo,Metro Station,Breakfast Spot,Sporting Goods Shop
6,Sheepshead Bay,Dessert Shop,Turkish Restaurant,Sandwich Place,Boat or Ferry,Miscellaneous Shop,Pizza Place,Buffet,Café,Chinese Restaurant,Karaoke Bar
9,Crown Heights,Pizza Place,Museum,Café,Bagel Shop,Coffee Shop,Bus Station,Bookstore,Supermarket,Candy Store,Sushi Restaurant
12,Windsor Terrace,Diner,Plaza,Café,Deli / Bodega,Grocery Store,Park,Bagel Shop,Moving Target,Middle Eastern Restaurant,Butcher
13,Prospect Heights,Bar,Thai Restaurant,Wine Shop,Mexican Restaurant,Cocktail Bar,Café,Pizza Place,Gourmet Shop,Diner,Coffee Shop
14,Brownsville,Restaurant,Moving Target,Chinese Restaurant,Park,Deli / Bodega,Burger Joint,Fried Chicken Joint,Caribbean Restaurant,Performing Arts Venue,Spanish Restaurant
15,Williamsburg,Bar,Coffee Shop,Pizza Place,Bagel Shop,Middle Eastern Restaurant,Latin American Restaurant,Tapas Restaurant,Taco Place,Liquor Store,Lounge


In [100]:
#Cluster 5
brooklyn_merged.loc[brooklyn_merged['Cluster Labels'] == 4, brooklyn_merged.columns[[1] + list(range(5, brooklyn_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
10,East Flatbush,Print Shop,Hardware Store,Pharmacy,Park,Chinese Restaurant,Caribbean Restaurant,Moving Target,Food & Drink Shop,Supermarket,Department Store
26,East New York,Deli / Bodega,Pharmacy,Metro Station,Fast Food Restaurant,Caribbean Restaurant,Event Service,Chinese Restaurant,Gym,Spanish Restaurant,Convenience Store
28,Canarsie,Grocery Store,Gym,Caribbean Restaurant,Chinese Restaurant,Asian Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Field
29,Flatlands,Pharmacy,Fast Food Restaurant,Discount Store,Fried Chicken Joint,Caribbean Restaurant,Nightclub,Park,Asian Restaurant,Chinese Restaurant,Bar
32,Coney Island,Baseball Stadium,Caribbean Restaurant,Theme Park Ride / Attraction,Brewery,Dessert Shop,Music Venue,Skating Rink,Café,Beach,Monument / Landmark
47,Prospect Park South,Caribbean Restaurant,Grocery Store,Pizza Place,Fast Food Restaurant,Mobile Phone Shop,Pharmacy,Donut Shop,Fried Chicken Joint,Latin American Restaurant,Mexican Restaurant
56,Rugby,Caribbean Restaurant,Supermarket,Bank,Pizza Place,Gym,Bus Station,Bus Stop,Mobile Phone Shop,Fried Chicken Joint,Seafood Restaurant
57,Remsen Village,Caribbean Restaurant,Fast Food Restaurant,Fish Market,Sandwich Place,Salad Place,Café,Liquor Store,Breakfast Spot,Supermarket,Fried Chicken Joint
69,Erasmus,Caribbean Restaurant,Yoga Studio,Grocery Store,Playground,Pizza Place,Convenience Store,Pharmacy,Donut Shop,School,Music Venue


## B. QUEENS

In [101]:
queens_data = neighborhoods[neighborhoods['Borough'] == 'Queens'].reset_index(drop=True)
queens_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Queens,Astoria,40.768509,-73.915654
1,Queens,Woodside,40.746349,-73.901842
2,Queens,Jackson Heights,40.751981,-73.882821
3,Queens,Elmhurst,40.744049,-73.881656
4,Queens,Howard Beach,40.654225,-73.838138


In [103]:
address = 'Queens, NY'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Queens are {}, {}.'.format(latitude, longitude))

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: Using Nominatim with the default "geopy/1.20.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  app.launch_new_instance()


The geograpical coordinate of Queens are 40.7498243, -73.7976337.


In [104]:
# create map of Manhattan using latitude and longitude values
map_queens = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(queens_data['Latitude'], queens_data['Longitude'], queens_data['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_queens)  
    
map_queens

### B1. The FourSquare API

In [105]:
CLIENT_ID = 'NNKKGPUSITWUJZNV5GJUHKUSPHL1MCGOTTWBDEHCOP01IRGV' # your Foursquare ID
CLIENT_SECRET = 'QM2K10LNRU40U3CAVO1UM1DDAW1BDK0HDZPKDGL2PL1YE3MQ' # your Foursquare Secret
VERSION = '20200108' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: NNKKGPUSITWUJZNV5GJUHKUSPHL1MCGOTTWBDEHCOP01IRGV
CLIENT_SECRET:QM2K10LNRU40U3CAVO1UM1DDAW1BDK0HDZPKDGL2PL1YE3MQ


In [113]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [114]:
queens_venues = getNearbyVenues(names=queens_data['Neighborhood'],
                                   latitudes=queens_data['Latitude'],
                                   longitudes=queens_data['Longitude']
                                  )

Astoria
Woodside
Jackson Heights
Elmhurst
Howard Beach
Corona
Forest Hills
Kew Gardens
Richmond Hill
Flushing
Long Island City
Sunnyside
East Elmhurst
Maspeth
Ridgewood
Glendale
Rego Park
Woodhaven
Ozone Park
South Ozone Park
College Point
Whitestone
Bayside
Auburndale
Little Neck
Douglaston
Glen Oaks
Bellerose
Kew Gardens Hills
Fresh Meadows
Briarwood
Jamaica Center
Oakland Gardens
Queens Village
Hollis
South Jamaica
St. Albans
Rochdale
Springfield Gardens
Cambria Heights
Rosedale
Far Rockaway
Broad Channel
Breezy Point
Steinway
Beechhurst
Bay Terrace
Edgemere
Arverne
Rockaway Beach
Neponsit
Murray Hill
Floral Park
Holliswood
Jamaica Estates
Queensboro Hill
Hillcrest
Ravenswood
Lindenwood
Laurelton
Lefrak City
Belle Harbor
Rockaway Park
Somerville
Brookville
Bellaire
North Corona
Forest Hills Gardens
Jamaica Hills
Utopia
Pomonok
Astoria Heights
Hunters Point
Sunnyside Gardens
Blissville
Roxbury
Middle Village
Malba
Hammels
Bayswater
Queensbridge


In [115]:
print(queens_venues.shape)
queens_venues.head()

(2128, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Astoria,40.768509,-73.915654,Favela Grill,40.767348,-73.917897,Brazilian Restaurant
1,Astoria,40.768509,-73.915654,Orange Blossom,40.769856,-73.917012,Gourmet Shop
2,Astoria,40.768509,-73.915654,Titan Foods Inc.,40.769198,-73.919253,Gourmet Shop
3,Astoria,40.768509,-73.915654,CrossFit Queens,40.769404,-73.918977,Gym
4,Astoria,40.768509,-73.915654,Simply Fit Astoria,40.769114,-73.912403,Gym


In [116]:
queens_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Arverne,17,17,17,17,17,17
Astoria,100,100,100,100,100,100
Astoria Heights,13,13,13,13,13,13
Auburndale,18,18,18,18,18,18
Bay Terrace,37,37,37,37,37,37
Bayside,72,72,72,72,72,72
Bayswater,2,2,2,2,2,2
Beechhurst,13,13,13,13,13,13
Bellaire,13,13,13,13,13,13


In [117]:
print('There are {} uniques categories.'.format(len(queens_venues['Venue Category'].unique())))

There are 272 uniques categories.


### B2. Analyze Each Neighborhood

In [118]:
# one hot encoding
queens_onehot = pd.get_dummies(queens_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
queens_onehot['Neighborhood'] = queens_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [queens_onehot.columns[-1]] + list(queens_onehot.columns[:-1])
queens_onehot = queens_onehot[fixed_columns]

queens_onehot.head()

,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Nail Salon,Neighborhood,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Nightlife,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Print Shop,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,0,0,0

In [119]:
queens_onehot.shape

(2128, 272)

In [120]:
queens_grouped = queens_onehot.groupby('Neighborhood').mean().reset_index()
queens_grouped

,Neighborhood,Yoga Studio,Accessories Store,Afghan Restaurant,American Restaurant,Antique Shop,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Athletics & Sports,Automotive Shop,BBQ Joint,Bagel Shop,Bakery,Bank,Bar,Baseball Field,Basketball Court,Bath House,Beach,Beach Bar,Beer Garden,Bike Trail,Bistro,Board Shop,Bookstore,Border Crossing,Boutique,Bowling Alley,Boxing Gym,Brazilian Restaurant,Breakfast Spot,Brewery,Bubble Tea Shop,Buffet,Burger Joint,Burrito Place,Bus Line,Bus Station,Bus Stop,Business Service,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Check Cashing Service,Chinese Restaurant,Church,Climbing Gym,Clothing Store,Cocktail Bar,Coffee Shop,College Academic Building,College Basketball Court,College Cafeteria,Colombian Restaurant,Comedy Club,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Cuban Restaurant,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Dive Bar,Dog Run,Donut Shop,Dosa Place,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Stand,Food Truck,French Restaurant,Fried Chicken Joint,Frozen Yogurt Shop,Fruit & Vegetable Store,Furniture / Home Store,Gas Station,Gastropub,Gay Bar,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Halal Restaurant,Harbor / Marina,Hardware Store,Health & Beauty Service,Health Food Store,Himalayan Restaurant,History Museum,Hobby Shop,Home Service,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Indoor Play Area,Intersection,Irish Pub,Italian Restaurant,Japanese Restaurant,Jewelry Store,Juice Bar,Karaoke Bar,Kids Store,Korean Restaurant,Kosher Restaurant,Lake,Latin American Restaurant,Laundromat,Laundry Service,Lingerie Store,Liquor Store,Lounge,Malay Restaurant,Market,Martial Arts Dojo,Massage Studio,Mattress Store,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Monument / Landmark,Motel,Motorcycle Shop,Movie Theater,Moving Target,Multiplex,Museum,Nail Salon,New American Restaurant,Nightclub,Noodle House,Office,Optical Shop,Other Nightlife,Paper / Office Supplies Store,Park,Pedestrian Plaza,Performing Arts Venue,Peruvian Restaurant,Pet Service,Pet Store,Pharmacy,Pier,Pizza Place,Platform,Playground,Plaza,Poke Place,Polish Restaurant,Pool Hall,Post Office,Print Shop,Pub,Ramen Restaurant,Rental Car Location,Rental Service,Residential Building (Apartment / Condo),Resort,Rest Area,Restaurant,Romanian Restaurant,Roof Deck,Salad Place,Salon / Barbershop,Sandwich Place,Scenic Lookout,School,Sculpture Garden,Seafood Restaurant,Shabu-Shabu Restaurant,Shanghai Restaurant,Shipping Store,Shoe Store,Shop & Service,Shopping Mall,Skate Park,Skating Rink,Smoke Shop,Snack Place,South American Restaurant,Southern / Soul Food Restaurant,Souvlaki Shop,Spa,Spanish Restaurant,Sporting Goods Shop,Sports Bar,State / Provincial Park,Steakhouse,Storage Facility,Strip Club,Supermarket,Supplement Shop,Surf Spot,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restaurant,Tapas Restaurant,Tea Room,Tennis Court,Thai Restaurant,Theater,Thrift / Vintage Store,Tibetan Restaurant,Toy / Game Store,Trail,Train,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Weight Loss Center,Wine Bar,Wine Shop,Wings Joint,Women's Store
0,Arver

In [121]:
queens_grouped.shape

(81, 272)

In [122]:
num_top_venues = 5

for hood in queens_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = queens_grouped[queens_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arverne----
             venue  freq
0        Surf Spot  0.24
1   Sandwich Place  0.12
2    Metro Station  0.12
3  Thai Restaurant  0.06
4       Playground  0.06


----Astoria----
                       venue  freq
0                        Bar  0.07
1  Middle Eastern Restaurant  0.06
2                 Hookah Bar  0.06
3           Greek Restaurant  0.05
4         Seafood Restaurant  0.04


----Astoria Heights----
           venue  freq
0   Burger Joint  0.08
1          Plaza  0.08
2         Bakery  0.08
3  Bowling Alley  0.08
4    Supermarket  0.08


----Auburndale----
                venue  freq
0  Miscellaneous Shop  0.06
1          Hookah Bar  0.06
2      Discount Store  0.06
3            Pharmacy  0.06
4           Pet Store  0.06


----Bay Terrace----
            venue  freq
0  Clothing Store  0.11
1   Women's Store  0.05
2      Kids Store  0.05
3  Cosmetics Shop  0.05
4  Lingerie Store  0.05


----Bayside----
                 venue  freq
0                  Bar  0.07
1     Sushi

              venue  freq
0            Bakery  0.08
1  Department Store  0.08
2    Cosmetics Shop  0.08
3       Supermarket  0.04
4      Liquor Store  0.04


----Lindenwood----
                     venue  freq
0  Fruit & Vegetable Store  0.11
1            Deli / Bodega  0.11
2                      Gym  0.11
3      Japanese Restaurant  0.11
4                    Hotel  0.11


----Little Neck----
                venue  freq
0       Deli / Bodega  0.06
1  Italian Restaurant  0.06
2  Chinese Restaurant  0.06
3   Korean Restaurant  0.04
4                Bank  0.04


----Long Island City----
                venue  freq
0         Coffee Shop  0.11
1               Hotel  0.11
2         Pizza Place  0.06
3  Mexican Restaurant  0.06
4                 Bar  0.06


----Malba----
                           venue  freq
0      Latin American Restaurant  0.25
1                      Rest Area  0.25
2                   Tennis Court  0.25
3  Vegetarian / Vegan Restaurant  0.25
4                    Yoga Stu

In [123]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [124]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = queens_grouped['Neighborhood']

for ind in np.arange(queens_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(queens_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Arverne,Surf Spot,Sandwich Place,Metro Station,Wine Shop,Pizza Place,Beach,Playground,Thai Restaurant,Bus Stop,Board Shop
1,Astoria,Bar,Hookah Bar,Middle Eastern Restaurant,Greek Restaurant,Seafood Restaurant,Bakery,Mediterranean Restaurant,Coffee Shop,Café,Italian Restaurant
2,Astoria Heights,Playground,Hostel,Pizza Place,Plaza,Bus Station,Burger Joint,Bowling Alley,Shopping Mall,Business Service,Italian Restaurant
3,Auburndale,Hookah Bar,Furniture / Home Store,Fast Food Restaurant,Athletics & Sports,Toy / Game Store,Noodle House,Train,Discount Store,Mobile Phone Shop,Miscellaneous Shop
4,Bay Terrace,Clothing Store,Women's Store,American Restaurant,Lingerie Store,Donut Shop,Mobile Phone Shop,Cosmetics Shop,Kids Store,Deli / Bodega,Coffee Shop


### B3. Cluster Neighborhoods

In [125]:
# set number of clusters
kclusters = 5

queens_grouped_clustering = queens_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(queens_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([2, 2, 0, 2, 2, 2, 3, 0, 2, 2], dtype=int32)

In [126]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

queens_merged = queens_data

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
queens_merged = queens_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

queens_merged.head() # check the last columns!

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Queens,Astoria,40.768509,-73.915654,2,Bar,Hookah Bar,Middle Eastern Restaurant,Greek Restaurant,Seafood Restaurant,Bakery,Mediterranean Restaurant,Coffee Shop,Café,Italian Restaurant
1,Queens,Woodside,40.746349,-73.901842,2,Grocery Store,Latin American Restaurant,Filipino Restaurant,Bakery,Thai Restaurant,American Restaurant,Pizza Place,Donut Shop,Pub,Deli / Bodega
2,Queens,Jackson Heights,40.751981,-73.882821,2,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Diner,Thai Restaurant,Mexican Restaurant,Empanada Restaurant,Clothing Store
3,Queens,Elmhurst,40.744049,-73.881656,2,Thai Restaurant,Mexican Restaurant,Chinese Restaurant,South American Restaurant,Vietnamese Restaurant,Gym / Fitness Center,Sushi Restaurant,Malay Restaurant,Donut Shop,Bar
4,Queens,Howard Beach,40.654225,-73.838138,2,Italian Restaurant,Bagel Shop,Pharmacy,Bank,Spa,Chinese Restaurant,Sandwich Place,Fast Food Restaurant,Hobby Shop,Shipping Store


In [127]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(queens_merged['Latitude'], queens_merged['Longitude'], queens_merged['Neighborhood'], queens_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### B4. Examine Clusters

In [128]:
# Cluster 1
queens_merged.loc[queens_merged['Cluster Labels'] == 0, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Richmond Hill,Pizza Place,Lounge,Caribbean Restaurant,Latin American Restaurant,Bank,Spanish Restaurant,Gym / Fitness Center,Supermarket,Sandwich Place,Clothing Store
12,East Elmhurst,Donut Shop,Coffee Shop,Hotel Bar,American Restaurant,Ice Cream Shop,Lake,Pizza Place,Rental Car Location,Gas Station,Supermarket
13,Maspeth,Pizza Place,Diner,Mobile Phone Shop,Bank,Grocery Store,Japanese Restaurant,Flower Shop,Sports Bar,Lounge,Sandwich Place
15,Glendale,Brewery,Food & Drink Shop,Pizza Place,Beer Garden,Arts & Crafts Store,Women's Store,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market
17,Woodhaven,Deli / Bodega,Pizza Place,Pharmacy,Bank,Bagel Shop,Dive Bar,Donut Shop,Restaurant,Nail Salon,Sandwich Place
18,Ozone Park,Pharmacy,Pizza Place,Diner,Gym,Bank,Breakfast Spot,Sporting Goods Shop,Tailor Shop,Restaurant,Latin American Restaurant
27,Bellerose,Pizza Place,Italian Restaurant,Flower Shop,Diner,Seafood Restaurant,Salon / Barbershop,Liquor Store,Bank,Donut Shop,Gym
28,Kew Gardens Hills,Bank,Pizza Place,Playground,Bus Station,Chinese Restaurant,Bagel Shop,Food,Middle Eastern Restaurant,Supermarket,Sushi Restaurant
30,Briarwood,Deli / Bodega,Arts & Crafts Store,Plaza,Convenience Store,Playground,Coffee Shop,Fast Food Restaurant,Bus Station,Gym,Mexican Restaurant
41,Far Rockaway,Deli / Bodega,Pizza Place,Pharmacy,Chinese Restaurant,Bank,Caribbean Restaurant,Bus Station,Fried Chicken Joint,Supermarket,Grocery Store


In [129]:
# Cluster 2
queens_merged.loc[queens_merged['Cluster Labels'] == 1, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
43,Breezy Point,Monument / Landmark,Trail,Beach,Women's Store,Farmers Market,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant
50,Neponsit,Beach,Lounge,Women's Store,Filipino Restaurant,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Empanada Restaurant


In [130]:
# Cluster 3
queens_merged.loc[queens_merged['Cluster Labels'] == 2, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Astoria,Bar,Hookah Bar,Middle Eastern Restaurant,Greek Restaurant,Seafood Restaurant,Bakery,Mediterranean Restaurant,Coffee Shop,Café,Italian Restaurant
1,Woodside,Grocery Store,Latin American Restaurant,Filipino Restaurant,Bakery,Thai Restaurant,American Restaurant,Pizza Place,Donut Shop,Pub,Deli / Bodega
2,Jackson Heights,Latin American Restaurant,Peruvian Restaurant,South American Restaurant,Bakery,Mobile Phone Shop,Diner,Thai Restaurant,Mexican Restaurant,Empanada Restaurant,Clothing Store
3,Elmhurst,Thai Restaurant,Mexican Restaurant,Chinese Restaurant,South American Restaurant,Vietnamese Restaurant,Gym / Fitness Center,Sushi Restaurant,Malay Restaurant,Donut Shop,Bar
4,Howard Beach,Italian Restaurant,Bagel Shop,Pharmacy,Bank,Spa,Chinese Restaurant,Sandwich Place,Fast Food Restaurant,Hobby Shop,Shipping Store
5,Corona,Mexican Restaurant,Church,Park,Playground,Supermarket,Empanada Restaurant,Sandwich Place,Donut Shop,Bakery,Restaurant
6,Forest Hills,Gym,Gym / Fitness Center,Yoga Studio,Thai Restaurant,Pharmacy,Park,Convenience Store,Pizza Place,Supermarket,Supplement Shop
7,Kew Gardens,Chinese Restaurant,Cosmetics Shop,Deli / Bodega,Donut Shop,Juice Bar,Bar,Bank,Indian Restaurant,Park,Pizza Place
9,Flushing,Hotpot Restaurant,Bubble Tea Shop,Chinese Restaurant,Korean Restaurant,Asian Restaurant,Construction & Landscaping,Gym / Fitness Center,Hotel,Gym,Bakery
10,Long Island City,Coffee Shop,Hotel,Mexican Restaurant,Pizza Place,Bar,Café,Donut Shop,Deli / Bodega,Chinese Restaurant,Gym / Fitness Center


In [131]:
# Cluster 4 
queens_merged.loc[queens_merged['Cluster Labels'] == 3, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
63,Somerville,Park,Fast Food Restaurant,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Women's Store,Egyptian Restaurant
79,Bayswater,Park,Playground,Women's Store,Farmers Market,Electronics Store,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Fast Food Restaurant


In [132]:
# Cluster 5
queens_merged.loc[queens_merged['Cluster Labels'] == 4, queens_merged.columns[[1] + list(range(5, queens_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
64,Brookville,Deli / Bodega,Women's Store,Filipino Restaurant,Empanada Restaurant,Event Space,Falafel Restaurant,Farm,Farmers Market,Fast Food Restaurant,Fish & Chips Shop



In general, while all of these tools are useful, there is nothing like visiting the actual city, seeing the neighborhoods, and speaking with residents. If it's possible, an in-person visit is highly recommended before making a big move.